https://www.kaggle.com/xhlulu/ieee-fraud-xgboost-with-gpu-fit-in-40s

https://www.kaggle.com/inversion/ieee-simple-xgboost

In [1]:
import os

import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb

In [2]:
train_transaction = pd.read_csv('datasets/train_transaction.csv')
test_transaction = pd.read_csv('datasets/test_transaction.csv')

train_identity = pd.read_csv('datasets/train_identity.csv')
test_identity = pd.read_csv('datasets/test_identity.csv')

sample_submission = pd.read_csv('datasets/sample_submission.csv')

train = train_transaction.merge(train_identity, how='outer', on='TransactionID')
test = test_transaction.merge(test_identity, how='outer', on='TransactionID')

print(train.shape)
print(test.shape)

(590540, 434)
(506691, 433)


In [3]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [4]:
# to identify columns with 1 unique value
one_value_cols = [col for col in train.columns if train[col].nunique() <= 1]
one_value_cols_test = [col for col in test.columns if test[col].nunique() <= 1]
print(one_value_cols)
print(one_value_cols_test)

[]
['V107']


In [5]:
# to identify the columns with over 90% of null values.
many_null_cols = [col for col in train.columns if train[col].isnull().sum() / train.shape[0] > 0.9]
many_null_cols_test = [col for col in test.columns if test[col].isnull().sum() / test.shape[0] > 0.9]

print(many_null_cols)
print(many_null_cols_test)

['dist2', 'D7', 'id_07', 'id_08', 'id_18', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27']
['dist2', 'id_07', 'id_08', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27']


In [6]:
# identify near zero variance predictors, that is columns with 90% of the same value
big_top_value_cols = [col for col in train.columns if train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_value_cols_test = [col for col in test.columns if test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]

print('training set:', big_top_value_cols)
print('testing set:', big_top_value_cols_test)

training set: ['isFraud', 'dist2', 'C3', 'D7', 'V98', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123', 'V124', 'V125', 'V129', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V281', 'V284', 'V286', 'V290', 'V293', 'V295', 'V296', 'V297', 'V298', 'V299', 'V300', 'V301', 'V305', 'V309', 'V311', 'V316', 'V318', 'V319', 'V320', 'V321', 'id_07', 'id_08', 'id_18', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27']
testing set: ['dist2', 'C3', 'V14', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V55', 'V65', 'V66', 'V67', 'V68', 'V77', 'V86', 'V88', 'V89', 'V98', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123', 'V124', 'V125', 'V129', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V281', 'V284', 'V286', 'V290', 'V29

In [7]:
# to combine all 3 lists of train columns and 3 lists of test columns
cols_to_drop = list(set(many_null_cols + many_null_cols_test + big_top_value_cols + big_top_value_cols_test + one_value_cols+ one_value_cols_test))
print(cols_to_drop)

['isFraud', 'V316', 'V112', 'C3', 'dist2', 'V109', 'V111', 'V297', 'V305', 'V113', 'V117', 'V135', 'V114', 'V89', 'V309', 'V101', 'V298', 'V121', 'id_07', 'V136', 'V301', 'V28', 'V118', 'V296', 'V120', 'V300', 'V320', 'V14', 'id_23', 'V66', 'V281', 'V104', 'V105', 'V122', 'V102', 'V293', 'V123', 'V23', 'id_22', 'id_21', 'V110', 'V284', 'V115', 'V86', 'V318', 'V319', 'V125', 'V26', 'V55', 'V108', 'V133', 'V119', 'V106', 'V88', 'V67', 'id_08', 'V311', 'V137', 'id_18', 'V25', 'V286', 'V107', 'V290', 'V68', 'id_26', 'V129', 'V132', 'V295', 'id_25', 'V116', 'V321', 'V124', 'V103', 'id_27', 'id_24', 'D7', 'V27', 'V77', 'V134', 'V98', 'V65', 'V299', 'V24']


In [8]:
# to remove the target variable "isFraud" from the list
cols_to_drop.remove('isFraud')
len(cols_to_drop)

82

In [9]:
# drop the columns in the list above
train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

In [10]:
# to see the rest of the columns
print('training set:', train.columns.values)
print('testing set:', test.columns.values)

training set: ['TransactionID' 'isFraud' 'TransactionDT' 'TransactionAmt' 'ProductCD'
 'card1' 'card2' 'card3' 'card4' 'card5' 'card6' 'addr1' 'addr2' 'dist1'
 'P_emaildomain' 'R_emaildomain' 'C1' 'C2' 'C4' 'C5' 'C6' 'C7' 'C8' 'C9'
 'C10' 'C11' 'C12' 'C13' 'C14' 'D1' 'D2' 'D3' 'D4' 'D5' 'D6' 'D8' 'D9'
 'D10' 'D11' 'D12' 'D13' 'D14' 'D15' 'M1' 'M2' 'M3' 'M4' 'M5' 'M6' 'M7'
 'M8' 'M9' 'V1' 'V2' 'V3' 'V4' 'V5' 'V6' 'V7' 'V8' 'V9' 'V10' 'V11' 'V12'
 'V13' 'V15' 'V16' 'V17' 'V18' 'V19' 'V20' 'V21' 'V22' 'V29' 'V30' 'V31'
 'V32' 'V33' 'V34' 'V35' 'V36' 'V37' 'V38' 'V39' 'V40' 'V41' 'V42' 'V43'
 'V44' 'V45' 'V46' 'V47' 'V48' 'V49' 'V50' 'V51' 'V52' 'V53' 'V54' 'V56'
 'V57' 'V58' 'V59' 'V60' 'V61' 'V62' 'V63' 'V64' 'V69' 'V70' 'V71' 'V72'
 'V73' 'V74' 'V75' 'V76' 'V78' 'V79' 'V80' 'V81' 'V82' 'V83' 'V84' 'V85'
 'V87' 'V90' 'V91' 'V92' 'V93' 'V94' 'V95' 'V96' 'V97' 'V99' 'V100' 'V126'
 'V127' 'V128' 'V130' 'V131' 'V138' 'V139' 'V140' 'V141' 'V142' 'V143'
 'V144' 'V145' 'V146' 'V147' 'V148' 'V14

In [11]:
train.shape

(590540, 352)

In [12]:
test.shape

(506691, 351)

In [14]:
from sklearn.preprocessing import LabelEncoder
cat_cols = ['ProductCD', 'card4', 'card6', 'M4','P_emaildomain','R_emaildomain', 
           'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
          'id_12', 'id_15', 'id_16', 'id_27', 'id_28', 'id_29','id_30', 'id_31', 
           'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38','DeviceType', 'DeviceInfo']

for col in cat_cols:
    if col in train.columns:
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))  

In [15]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,4,13926,NaN,150.0,1,142.0,...,136,NaN,461,4,2,2,2,2,2,2740
1,2987001,0,86401,29.0,4,2755,404.0,150.0,2,102.0,...,136,NaN,461,4,2,2,2,2,2,2740
2,2987002,0,86469,59.0,4,4663,490.0,150.0,4,166.0,...,136,NaN,461,4,2,2,2,2,2,2740
3,2987003,0,86499,50.0,4,18132,567.0,150.0,2,117.0,...,136,NaN,461,4,2,2,2,2,2,2740
4,2987004,0,86506,50.0,1,4497,514.0,150.0,2,102.0,...,162,32.0,268,3,1,0,1,1,1,1565


In [16]:
test.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,3663549,18403224,31.95,4,10409,111.0,150.0,4,226.0,2,...,136,NaN,461,4,2,2,2,2,2,2740
1,3663550,18403263,49.00,4,4272,111.0,150.0,4,226.0,2,...,136,NaN,461,4,2,2,2,2,2,2740
2,3663551,18403310,171.00,4,4476,574.0,150.0,4,226.0,2,...,136,NaN,461,4,2,2,2,2,2,2740
3,3663552,18403310,284.95,4,10989,360.0,150.0,4,166.0,2,...,136,NaN,461,4,2,2,2,2,2,2740
4,3663553,18403317,67.95,4,18018,452.0,150.0,2,117.0,2,...,136,NaN,461,4,2,2,2,2,2,2740


In [22]:
train.fillna(-999, inplace=True)
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,4,13926,-999.0,150.0,1,142.0,...,136,-999.0,461,4,2,2,2,2,2,2740
1,2987001,0,86401,29.0,4,2755,404.0,150.0,2,102.0,...,136,-999.0,461,4,2,2,2,2,2,2740
2,2987002,0,86469,59.0,4,4663,490.0,150.0,4,166.0,...,136,-999.0,461,4,2,2,2,2,2,2740
3,2987003,0,86499,50.0,4,18132,567.0,150.0,2,117.0,...,136,-999.0,461,4,2,2,2,2,2,2740
4,2987004,0,86506,50.0,1,4497,514.0,150.0,2,102.0,...,162,32.0,268,3,1,0,1,1,1,1565


In [23]:
test.fillna(-999, inplace=True)
test.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,3663549,18403224,31.95,4,10409,111.0,150.0,4,226.0,2,...,136,-999.0,461,4,2,2,2,2,2,2740
1,3663550,18403263,49.00,4,4272,111.0,150.0,4,226.0,2,...,136,-999.0,461,4,2,2,2,2,2,2740
2,3663551,18403310,171.00,4,4476,574.0,150.0,4,226.0,2,...,136,-999.0,461,4,2,2,2,2,2,2740
3,3663552,18403310,284.95,4,10989,360.0,150.0,4,166.0,2,...,136,-999.0,461,4,2,2,2,2,2,2740
4,3663553,18403317,67.95,4,18018,452.0,150.0,2,117.0,2,...,136,-999.0,461,4,2,2,2,2,2,2740


In [24]:
# Save the data for ML
train.to_csv('/Users/wanranli/Downloads/!Capstone Project 2/datasets/train_ML.csv')

In [25]:
# Save the data for ML
train.to_csv('/Users/wanranli/Downloads/!Capstone Project 2/datasets/test_ML.csv')